In [1]:
%load_ext autoreload

In [2]:
import sys

if '../' not in sys.path:
    sys.path.append('../')

In [3]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from fractions import Fraction

In [4]:
%autoreload
from fxtrade.keyhold import KeyHold

key_config = KeyHold()
key_config

'/home/jovyan/.config/keyhold/keys.json'
{
  "yahoo": {
    "x-api-key": "mz4P************************************"
  },
  "cryptowatch": {
    "x-cw-api-key": "WLWU****************",
    "api_secret": "TOFt************************************"
  },
  "bitflyer": {
    "api_key": "ULkE******************",
    "api_secret": "IYGp****************************************"
  },
  "twitter": {
    "consumer_key": "pt4B*********************",
    "consumer_secret": "5ciB**********************************************",
    "api_key": "1218**********************************************",
    "api_secret": "MJJk*****************************************",
    "bearer_token": "AAAA************************************************************************************************************"
  }
}

In [5]:
%autoreload

from fxtrade.api import CodePair, CrangePeriod
from fxtrade.stock import Stock, Rate
from fxtrade.trade import Trade, TradePair, History, Report
from fxtrade.stocks import JPY, BTC

from fxtrade.interface.cryptowatch import CryptowatchAPI
from fxtrade.interface.bitflyer import BitflyerAPI

from fxtrade.wallet import Wallet
from fxtrade.chart import ChartDummyAPI, Board, Chart
from fxtrade.trader import TraderDummyAPI, Trader
from fxtrade.fx import FX

In [6]:
# チャート取得のための API を指定
chart_api = CryptowatchAPI(api_key=key_config['cryptowatch']['x-cw-api-key'])

# チャートを管理するクラスの生成
chart = Chart(chart_api,
              code_pair=CodePair('JPY', 'BTC'),
              data_dir='../data/chart',
              crange_period=[CrangePeriod('max', '15m')], #crange_interval=['10y-1d', '1mo-15m', '5d-1m']
             )

In [7]:
# 取引のための API を指定
trader_api = BitflyerAPI(api_key=key_config['bitflyer']['api_key'],
                        api_secret=key_config['bitflyer']['api_secret'])

#hist = trade_api.get_history(start_date=pd.Timestamp(2021, 12, 1))

# 取引を管理するクラスの生成
trader = Trader(
            api=trader_api,
            code_pair=CodePair('JPY', 'USD'),
            chart=chart,
            history=None,
            data_dir='../data/trader',
         )

In [8]:
fx = FX(name='trader1', origin='JPY', chart_api=chart_api, trader_api=trader_api, data_dir='../data')

In [9]:
fx.wallet == Wallet()

True

In [10]:
fx.generate_client('BTC', crange_period=[CrangePeriod('max', '15m')])

In [11]:
fx

FX(name='trader1',
    origin='JPY',
    data_dir='../data/trader1',
    markets={
        'BTC': Trader(api=BitflyerAPI(api_key='ULkE...', api_secret='IYGp...'),
            code_pair=CodePair(base='BTC', quote='JPY'),
            data_dir='../data/trader1/trader',
            wallet=Wallet({
                'BTC': Stock(BTC, 0),
                'JPY': Stock(JPY, 0),
            }),
            chart=Chart(api=CryptowatchAPI(api_key='WLWU...'),
                code_pair=CodePair(base='BTC', quote='JPY'),
                data_dir='../data/trader1/chart',
                crange_period=[CrangePeriod(crange='max', period='15m')],
                board={
                    'CrangePeriod(crange='max', period='15m')': Board(api=CryptowatchAPI(api_key='WLWU...'),
                        code_pair='CodePair(base='BTC', quote='JPY')',
                        crange_period='CrangePeriod(crange='max', period='15m')',
                        interval=None,
                        first_updated=No

In [12]:
fx.create_emulator(name='emulator1')

FX(name='emulator1',
    origin='JPY',
    data_dir='../data/emulator1',
    markets={
        'BTC': Trader(api=TraderEmulatorAPI(api=BitflyerAPI, source_dir='../data/trader1/trader'),
            code_pair=CodePair(base='BTC', quote='JPY'),
            data_dir='../data/emulator1/trader',
            wallet=Wallet({
                'BTC': Stock(BTC, 0),
                'JPY': Stock(JPY, 0),
            }),
            chart=Chart(api=ChartEmulatorAPI(api=CryptowatchAPI, source_dir='../data/trader1/chart'),
                code_pair=CodePair(base='BTC', quote='JPY'),
                data_dir='../data/emulator1/chart',
                crange_period=[CrangePeriod(crange='max', period='15m')],
                board={
                    'CrangePeriod(crange='max', period='15m')': Board(api=ChartEmulatorAPI(api=CryptowatchAPI, source_dir='../data/trader1/chart'),
                        code_pair='CodePair(base='BTC', quote='JPY')',
                        crange_period='CrangePeriod(cran

In [13]:
fx.logger.info("hoge")

| trader1 | 2023-02-08 01:00:51 +0900 | INFO : hoge |


In [14]:
fx.wallet

Wallet({'BTC': Stock(BTC, 0), 'JPY': Stock(JPY, 0)})

In [15]:
fx.save_wallet(append=True)

| trader1 | 2023-02-08 01:00:51 +0900 | INFO : FX['BTC'] save_wallet() -> '../data/trader1/trader/wallet.csv'. |


In [16]:
fx.save_wallet([])

| trader1 | 2023-02-08 01:00:51 +0900 | INFO : FX save_wallet() ... no trader to save. |


In [17]:
fx['BTC'].read_wallet(return_t=True)

(Wallet({'BTC': Stock(BTC, 0), 'JPY': Stock(JPY, 0)}),
 Timestamp('2023-02-08 01:00:52'))

In [18]:
fx['BTC'].wallet

Wallet({'BTC': Stock(BTC, 0), 'JPY': Stock(JPY, 0)})

In [19]:
fx['BTC'].load_wallet()

Wallet({'BTC': Stock(BTC, 0), 'JPY': Stock(JPY, 0)})

In [20]:
fx['BTC'].wallet

Wallet({'BTC': Stock(BTC, 0), 'JPY': Stock(JPY, 0)})

In [21]:
fx['BTC'].download_wallet()

Wallet({'JPY': Stock(JPY, 115411), 'BTC': Stock(BTC, 33/50000000), 'BCH': Stock(BCH, 0), 'ETH': Stock(ETH, 0), 'ETC': Stock(ETC, 0), 'LTC': Stock(LTC, 0), 'MONA': Stock(MONA, 0), 'LSK': Stock(LSK, 0), 'XRP': Stock(XRP, 0), 'BAT': Stock(BAT, 0), 'XLM': Stock(XLM, 0), 'XEM': Stock(XEM, 0), 'XTZ': Stock(XTZ, 0), 'DOT': Stock(DOT, 0), 'LINK': Stock(LINK, 0), 'XYM': Stock(XYM, 0), 'MATIC': Stock(MATIC, 0), 'MKR': Stock(MKR, 0), 'ZPG': Stock(ZPG, 0), 'FLR': Stock(FLR, 0)})

次、Wallet のテスト書くとこから

In [22]:
raise

RuntimeError: No active exception to reraise

In [ ]:
#fx['BTC'].chart['max-15m'].download()

In [ ]:
chart = Chart(api=ChartDummyAPI(),
          code_pair=CodePair('BTC', 'JPY'),
          data_dir='../data/chart',
          crange_period=[CrangePeriod('max', '15m')],
        )

In [ ]:
chart._to_crange_period_dict([CrangePeriod('max', '15m')])

In [ ]:
chart._to_crange_period_dict({
    'max-15m': ...,
    CrangePeriod('max', '1m'): ...,
})

In [ ]:
from fxtrade.logger import Logger

In [ ]:
logger = Logger('myapp', './log/log')

In [ ]:
logger._logger

In [ ]:
logger._logger.debug('hoge')

In [ ]:
logger._logger.warning('hoge')

In [ ]:
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

In [ ]:
from datetime import datetime
datetime.now()

# ダウンロードのとこから

In [ ]:
raise

In [ ]:
# チャート取得のための API を指定
chart_api = YahooFinanceAPI(api_key=key_config['yahoo']['x-api-key'])

# チャートを管理するクラスの生成
chart = Chart(
            ticker='BTC-JPY',
            api=chart_api,
            data_dir='../data/chart',
        )

In [ ]:
# 取引のための API を指定
trade_api = BitflyerAPI(api_key=key_config['bitflyer']['api_key'],
                        api_secret=key_config['bitflyer']['api_secret'])

#hist = trade_api.get_history(start_date=pd.Timestamp(2021, 12, 1))

# 取引を管理するクラスの生成
trader = Trader(
            api=trade_api,
            history=None,
            data_dir='../data/trader',
         )

In [ ]:
wallet = Wallet(['JPY', 'BTC'])
wallet

In [ ]:
# 取引アルゴリズムを統括するためのクラスを生成
fx = FX(wallet=wallet, chart=chart, trader=trader)
fx

In [ ]:
fx.chart.dirmap

In [ ]:
fx.wallet

In [ ]:
fx.sync_wallet(codes={'JPY', 'BTC'})

In [ ]:
fx.wallet

In [ ]:
fx

In [ ]:
hist = fx.trader.get_history(start_date=datetime(2022, 2, 1))

In [ ]:
hist.summarize().applymap(lambda x: x if not isinstance(x, Fraction) else float(x))

In [ ]:
trade = fx.get_max_available()
trade

In [ ]:
trade = fx.get_max_salable()
trade

In [ ]:
fx.get_last_trade()

In [ ]:
UPDATE = False

if UPDATE:
    fx.chart.update()
    fx.chart.save()
else:
    fx.chart.load()

In [ ]:
fx.chart.dfs

In [ ]:
fx = fx.create_emulator(root_dir='../data/emulator')

In [ ]:
fx.chart.dfs

In [ ]:
fx.chart.download('1mo-15m')

In [ ]:
fx.chart.flush()
fx.chart.dfs

# 確認

In [ ]:
#_ = fx.chart.update(t=datetime(2022, 7, 14, 17, 15))
_ = fx.chart.update(t=datetime(2022, 8, 20, 17, 15))

In [ ]:
fx.chart['10y-1d']

In [ ]:
fx.chart['5d-1m']

In [ ]:
fx.chart.normalize()

In [ ]:
%autoreload

from fxtrade.analysis import *

In [ ]:
chart = fx.chart.dfs['1mo-15m']

low = chart['low']
high = chart['high']
glow = emaverage(low, 0.87, pd.Timedelta(minutes=15))

In [ ]:
fx.chart.normalize('1mo-15m')

In [ ]:
df_a = analyze(fx.chart['1mo-15m']['low'])
df_a

In [ ]:
plot_laplace(df_a['diff'])

In [ ]:
estimate_probability(df_a['diff'], 1)

# 自動取引アルゴリズム

In [ ]:
%autoreload
from fxtrade.algorithm.tenet import calc_zpc, calc_best_alpha, Tenet

In [ ]:
fx.chart.normalize('1mo-15m')

df = analyze(fx.chart['1mo-15m']['low'])
df

In [ ]:
loc, scale = estimate_laplace_param(df['diff'].iloc[-100:])
loc, scale

In [ ]:
crange_interval = '1mo-15m'
dt = pd.Timedelta(minutes=15)

In [ ]:
tenet = Tenet(crange_interval=crange_interval, dt=dt, window=3, alpha=0.87)

In [ ]:
tenet.analyze(fx)

### 買い時
* アップトレンドかつ平均より下かつ暴落ではない
* アップトレンドかつ暴騰中（大きく張る）
* アップトレンドかつ暴落終了直後
* ダウントレンドかつ暴落終了後の暴騰（大きく張る）

### 売り時
* 暴騰終了直後
* 暴落開始

In [ ]:
tenet.plot()

In [ ]:
tenet.df_low.iloc[[-1]]

In [ ]:
trade = tenet(fx)
trade

In [ ]:
fx.apply(tenet)

In [ ]:
try:
    fx.order(trade)
except ValueError as e:
    print('No order was made:', e)

In [ ]:
trade

In [ ]:
fx.trader.chart

In [ ]:
#fx.sell(fx.get_max_salable())

In [ ]:
#fx.execute(tenet)

In [ ]:
#fx.buy(trade)

In [ ]:
#resp = fx.trader.api.buy(float(trade.y.q))
#resp

In [ ]:
# sel = tenet.sell(fx)

In [ ]:
# sel

In [ ]:
# fx.trader.api.sell(float(sel.x.q))